In [1]:
import csv
import os
import re

def courtHistory(listHasil):

	csvFolrder = "./"
	csvName = "courtHistory.csv"

	if os.path.exists(csvName):

		f = open(csvName, 'a', newline='\n')
		print(f)
		print("ada")
		w = csv.writer(f)

	else:

		f = open(csvName, 'w', newline='\n')
		print(f)
		print("tidak ada")
    
		w = csv.writer(f)
		w.writerow(('Nomor Putusan', 'Nama Terdakwa 1', 'Nama Terdakwa 2', 'Nama Terdakwa 3', 'Tuntutan Pidana', 'Tuntutan KUHP', 'Tuntukan Hukuman', 'Putusan Pidana', 'Putusan Hukuman', 'Tanggal Putusan', 'Hakim Ketua', 'Hakim Anggota1', 'Hakim Anggota2', 'Panitera', 'Penuntut Umum'))
    
	# menulis file csv
	for s in listHasil:
		w.writerow(s)

	f.close()
	berhasil = "\nCreate Csv file Berhasil\n" 
	return berhasil





In [2]:
import re

def generateEntity(pathFile,fileName):

    print("a")
    folderData = pathFile
    folderHasil = "./OUTPUT/"
    #namaFile = "12Pid.B2015.PNJktTim.txt"
    namaFile = fileName
    namaFileHasil = "O-"+namaFile
    listHasil =[]

    file_putusan = open(folderData+namaFile, "r", encoding='UTF8')
    baca_baris = file_putusan.readlines()

    terdakwa = 0
    tuntutan_pidana = 0
    tuntutanPidana = 0
    tuntutanHukuman = 0
    tindakPidana = 0
    hukuman = 0
    nomor_putusan = 0
    listTerdakwa=[]

    print("")
    print("====== KEPUTUSAN PENGADILAN =====")
    with open(folderHasil + namaFile, "w", encoding="utf-8") as file:

        for baris in baca_baris:
            panjang_baris = len(baris)
            if panjang_baris > 3:
                baris=baris.lower()

                # mendapatkan entitas NOMOR PUTUSAN
                if baris.find("pid.b") >= 0 and nomor_putusan == 0:
                    nomor_putusan +=1
                    eNomor = re.search( r'\d+/(.)*/\d{4}/(.)*',baris, re.M|re.I)	

                    print("Nomor Putusan : " + eNomor.group())
                    file.writelines("Nomor Putusan : " + eNomor.group() + "\n")
                    listHasil.append(eNomor.group().rstrip("\n")) 
                    
                # mendapatkan entitas NAMA TERDAKWA
                if baris.find("nama terdakwa :") >= 0 or baris.find("nama lengkap :") >= 0:
                    terdakwa += 1
                    eTerdakwa = re.search( r'nama (lengkap|terdakwa) :(.*)',baris, re.M|re.I)                
                    terdakwaKe = str(terdakwa)
                    print("Terdakwa Ke : "+terdakwaKe)
                    print("Nama Terdakwa : "+eTerdakwa.group(2))
                    file.writelines("Terdakwa Ke : "+terdakwaKe + "\n")
                    file.writelines("Nama Terdakwa : "+eTerdakwa.group(2) + "\n" + "\n")
                    listTerdakwa.append(eTerdakwa.group(2).rstrip("\n"))

                if (baris.find("terdakwa ditahan") >= 0 or baris.find("telah ditahan") >= 0 or baris.find("terdakwa berada") >= 0 )and terdakwa >= 1:
                    listHasil.append(listTerdakwa[0])
                    if terdakwa == 1: 
                        listHasil.append("")                
                        listHasil.append("")
                    if terdakwa == 2: 
                        listHasil.append(listTerdakwa[1])                
                        listHasil.append("")
                    if terdakwa == 3: 
                        listHasil.append(listTerdakwa[1])                
                        listHasil.append(listTerdakwa[2])
                    terdakwa = 0
                    
                # mendapatkan entitas TUNTUTAN PIDANA
                if baris.find("menyatakan terdakwa") >= 0 and tuntutan_pidana == 0 and tuntutanPidana==0:
                    print("====== TUNTUTAN =====")
                    file.writelines("TUNTUTAN" + "\n")
                    if baris.find("sebagaimana") >= 0:
                        ePidana = re.search( r'melakukan tindak pidana (.*) sebagaimana',baris, re.M|re.I)                
                    else:
                        ePidana = re.search( r'melakukan tindak pidana (.*) ;',baris, re.M|re.I)
                        
                    if ePidana:
                        print("Tindak Pidana : "+ePidana.group(1))    
                        file.writelines("Tindak Pidana : "+ePidana.group(1) + "\n")        
                        listHasil.append(ePidana.group(1))
                    else:
                        print("Tindak Pidana : ")            
                        listHasil.append("")                

                    eKUHP = re.search( r'pasal (.*) kuhp',baris, re.M|re.I)
                    if eKUHP: 
                        print("Melanggar KUHP : " +eKUHP.group())
                        file.writelines("Melanggar KUHP : " +eKUHP.group() + "\n")
                        listHasil.append(eKUHP.group())
                    else:
                        print("Melanggar KUHP : ")
                        file.writelines("Melanggar KUHP : " + "\n")
                        listHasil.append("")                    
                    tuntutanPidana = 1

                if baris.find("menjatuhkan pidana") >= 0 and tuntutan_pidana == 0 and tuntutanHukuman == 0:
                    #print(baris)
                    etuntutan = re.search( r'selama (.*) dikurangi',baris, re.M|re.I)
                    print("Tuntutan Hukuman : "+etuntutan.group(1))
                    file.writelines("Tuntutan Hukuman : "+etuntutan.group(1) + "\n")
                    listHasil.append(etuntutan.group(1))                
                    tuntutan_pidana = 1

                # mendapatkan entitas PUTUSAN PIDANA
                if baris.find("menyatakan terdakwa") >= 0 and baris.find("terbukti") >= 0 and tuntutan_pidana == 1 and tuntutanPidana == 1:
                    print("====== PUTUSAN =====")
                    print(baris)
                    if baris.find("sebagaimana") >= 0:
                        ePutPidana = re.search( r'melakukan tindak pidana (.*) sebagaimana',baris, re.M|re.I)
                    else:
                        ePutPidana = re.search( r'melakukan tindak pidana (.*) ;',baris, re.M|re.I)
                    print("Putusan Pidana : "+ePutPidana.group(1))        
                    file.writelines("Putusan Pidana : "+ePutPidana.group(1))    
                    listHasil.append(ePutPidana.group(1))
                    tuntutanHukuman = 1

                if baris.find("menjatuhkan pidana") >= 0 and tuntutan_pidana == 1 and tuntutanHukuman == 1:
                    #print(baris)
                    if baris.find("dikurangi") >= 0:
                        ePutusan = re.search( r'selama (.*) dikurangi',baris, re.M|re.I)
                    else:
                        ePutusan = re.search( r'selama (.*) ;',baris, re.M|re.I)    
                    print("Putusan Hukuman : "+ePutusan.group(1))
                    file.writelines("Putusan Hukuman : "+ePutusan.group(1))
                    listHasil.append(ePutusan.group(1))
                    #tuntutan_pidana = 1    
            
                # mendapatkan HAKIM KETUA MAJLIS HAKIM            
                if baris.find("diputuskan dalam") >= 0:
                    print("====== PENUTUP =====")
                    cari_tgl_putusan = baris.find("tanggal")
                    cari_tgl_putusan1 = baris.find("oleh")

                    if baris.find(", se") >= 0:
                        cari_hakim_ketua1 = baris.find(", se")
                    else:
                        cari_hakim_ketua1 = baris.find(". se")

                    if baris.find("oleh kami") >= 0:
                        oleh = 9
                    else:
                        oleh = 5

                    if baris.find("ketua majelis hakim, ") >= 0:  
                        cari_hakim_anggota1 = baris.find("ketua majelis hakim, ")+21
                    elif baris.find("hakim ketua majelis, ") >=0:    
                        cari_hakim_anggota1 = baris.find("hakim ketua majelis, ")+21
                    elif baris.find("hakim ketua, ") >=0:
                        cari_hakim_anggota1 = baris.find("hakim ketua, ")+13

                    cari_hakim_anggota2 = baris.find("masing")            
                    hakim_anggota = baris[cari_hakim_anggota1:cari_hakim_anggota2-1]
                    hakim_anggota1 = hakim_anggota[0:hakim_anggota.find(" dan")]
                    hakim_anggota2 = hakim_anggota[hakim_anggota.find(" dan")+5:len(hakim_anggota)]

                    if baris.find("dibantu oleh") >= 0:
                        cari_panitera1 = baris.find("dibantu oleh")+13
                    else:    
                        cari_panitera1 = baris.find("dibantu")+8

                    if baris.find("sebagai panitera pengganti") >= 0:              
                        cari_panitera2 = baris.find("sebagai panitera pengganti")
                    elif baris.find("panitera pengganti") >= 0:              
                        cari_panitera2 = baris.find("panitera pengganti")
                        
                    if baris.find("dihadiri oleh") >= 0:
                        cari_penuntut_umum1 = baris.find("dihadiri oleh")+13
                    elif baris.find("dihadiri") >= 0:
                        cari_penuntut_umum1 = baris.find("dihadiri")+9    

                    if baris.find("jaksa penuntut umum") >= 0:
                        cari_penuntut_umum2 = baris.find("jaksa penuntut umum")
                    elif baris.find("penuntut umum") >= 0:
                        cari_penuntut_umum2 = baris.find("penuntut umum")    
                    
                    tanggalPutusan = baris[cari_tgl_putusan+8:cari_tgl_putusan1]
                    hakimKetua = baris[cari_tgl_putusan1+oleh:cari_hakim_ketua1]
                    hakimAnggota1 = hakim_anggota1
                    hakimAnggota2 = hakim_anggota2
                    panitera = baris[cari_panitera1:cari_panitera2]
                    penuntutUmum = baris[cari_penuntut_umum1:cari_penuntut_umum2]

                    print("Tanggal Putusan : "+tanggalPutusan)
                    print("Hakim Ketua : "+hakimKetua)
                    print("Hakim Anggota 1 : "+hakimAnggota1)
                    print("Hakim Anggota 2 : "+hakimAnggota2)
                    print("Panitera : "+panitera)
                    print("Penuntut Umum : "+penuntutUmum)
                    
                    file.writelines("\n" + "\n" + "PUTUSAN" + "\n")
                    file.writelines("Tanggal Putusan : " + tanggalPutusan + "\n")
                    file.writelines("Hakim Ketua : " + hakimKetua + "\n")
                    file.writelines("Hakim Anggota 1 : "+hakimAnggota1 + "\n")
                    file.writelines("Hakim Anggota 2 : "+hakimAnggota2 + "\n")
                    file.writelines("Panitera : "+panitera + "\n")
                    file.writelines("Penuntut Umum : "+penuntutUmum + "\n")

                    listHasil.append(tanggalPutusan)
                    listHasil.append(hakimKetua)
                    listHasil.append(hakimAnggota1)
                    listHasil.append(hakimAnggota2)
                    listHasil.append(panitera)
                    listHasil.append(penuntutUmum)
        
            
            
    file_putusan.close()
    print("======================")
    return listHasil


# #start_path = "./INPUT/"
# #namaFile = "48Pid.B2015.PNJKTTIM.txt"

# #hasil = generateEntity(start_path, namaFile)
# #print(hasil)


In [3]:
import re

state_data = {
    0: "tidak ada",
    1: "terdakwa"
}

In [4]:
def cari_terdakwa(data):
    terdakwa = 1
    hasil_terdakwa = []
    data_arr = data.lower().split("\n")
    index_terdakwa = []
    
    if re.search("terdakwa " + str(terdakwa), data.lower()):
        while re.search("terdakwa " + str(terdakwa), data.lower()):
            terdakwa += 1
        
        terdakwa = 1
        for baris in data_arr:
            if re.search(r"nama (lengkap|terdakwa) :(.*)", baris):
                print(baris.split(":")[1][0:(len(baris.split(":")[1]) - 1)])
        
        
        
def generateEntity(pathFile,fileName):
    
    folderData = pathFile
    folderHasil = "./OUTPUT/"
    #namaFile = "12Pid.B2015.PNJktTim.txt"
    namaFile = fileName
    namaFileHasil = "O-"+namaFile
    listHasil = []
    
    state = state_data[0]
    
    file_putusan = open(folderData+namaFile, "r", encoding='UTF8')
    baca_baris = file_putusan.read()
    cari_terdakwa(baca_baris)
    
    # for baris in baca_baris:
    
    #     # print(baris.split("\n"))
    #     if baris.split("\n")[0] == "berikut dalam perkara Terdakwa :":
    #         if re.search("perkara terdakwa", baris.split("\n")[0].lower()):
    #             state = state_data[1]
    
    #         if state == state_data[1]:
    #             print("data")            
        
        # print(re.search("perkara terdakwa", baris.lower()))


# generateEntity("./INPUT/", "output2.txt")
    

In [5]:
import os
# from courtHistoryCsv import courtHistory

def main():

    start_path = "./INPUT/"
    hasil_path = "./OUTPUT/"
    daftarFile = os.listdir(start_path)
    listHasil = []

    # namaFile = "48Pid.B2015.PNJKTTIM.txt"
    # namaFile = "161Pid.B2015.PNJktTim.txt"
    # hasil = generateEntity(start_path, namaFile)
    # listHasil.append(hasil)

    for tiapFile in daftarFile:
        # print(tiapFile)
        hasil = generateEntity(start_path, tiapFile)
        listHasil.append(hasil)
        print("======= ROW HASIL =======")    
        # print(hasil)
        break


	
    print("======= HASIL AKHIR =======")
 

main()


======= ROW HASIL =======
======= HASIL AKHIR =======


In [6]:
def preproses(data):
    return data.lower().split("\n")

In [ ]:
def exctract_terdakwa(data, finder):
    pattern = [
        "perkara t erdakwa",
        "perkara"
    ]
    
    start = data.find(finder)    

    for i in pattern:
        if start != -1:
            break
        
        start = data.find(i)
    
 
    data = data[start:]
    nama_raw = data[data.find("nama lengkap") + len("nama lengkap : "):]
    nama_raw = nama_raw[:data.find(";")] + " "
    nama_raw = nama_raw[:data.find("tempat")] + " "
    nama_raw = nama_raw[:nama_raw.find("2")].replace(";", "")
    return nama_raw.strip()

def cari_terdakwa(data):
    data = " ".join(" ".join(preproses(data)).split())
    total_terdakwa = 0
    list_terdakwa = []
    while data.find("terdakwa " + str(total_terdakwa + 1)) != -1:
        total_terdakwa += 1
    if total_terdakwa < 1:
        list_terdakwa.append(exctract_terdakwa(data, "perkara terdakwa"))
    else:
        for i in range(1, total_terdakwa + 1):
            list_terdakwa.append((exctract_terdakwa(data, "terdakwa " + str(i))))
    
    return list_terdakwa
            
list_file = os.listdir("./INPUT/")

for filename in list_file:
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        print(cari_terdakwa(file.read()))
    

-1
['taufiq novi khamsy ah bin (alm) murjani']
40
['muhammad afifudin als koplek bin fauzan tempat lahir : pekalongan umur / tanggal lahir : 3']
-1
['wahid syiyam alias burung alias angga wijaya bin asnoto']
30
['harto alias penot bin cahyono tempat lahir : pekalongan umur/']
-1
['heppy kurniawan anak dari daryono']
-1
['ramli als mahedi bin (alm) abdurahman']
60
['gaffy tri pamungkas alias gaffy anak dari imam budi raharjo tempat lahir : pekalongan umur/tanggal lahir :']
43
['juli agus prasetyo alias julex bin karnoto tempat lahir : pekalongan umur/tanggal lahir :']
-1
['farid alias habib bin (alm) husain']
-1
['nur wahyo als gendut bin suhardi']
-1
['wa harto alias penot bin cahyono dengan nomor perkara nomor 309/pid.b/']
-1
['umar jamal maretan bin jamal ahmad maretan']
-1
['purwanto bin daudi']
-1
['moh. agus riyanto bin sakhudin']
-1
['nur wahyo als gendut bin suhardi']
52
['putra adhib harisanto alias adib bin paulus sunaryo tempat lahir : kulon progo umur/ tanggal lahir : 39']
-

In [98]:
def cari_tuntutan_tp(data):

    data = " ".join(data.lower().split())

    data = data[data.find("melakukan"):] + " "
    data = data[:data.find("sebagaimana")] + " "
    data = data[:data.find("sesuai")] + " "
    
    return " ".join(data.split()[3:]).replace("“", "").replace("”", "").replace("\"", "").strip()


list_file = os.listdir("./INPUT/")

for filename in list_file:
    data = ""
    
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()

    # print(filename)
    (cari_tuntutan_tp(data))
    print()
    # print()

In [97]:
def cari_putusan_tp(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    end = data.find("2.")
    data = data[:end]
    
    data = " ".join(data.lower().split())

    data = data[data.find("melakukan"):] + " "
    data = data[:data.find("sebagaimana")] + " "
    data = data[:data.find(" s ebagaimana")] + " "
    data = data[:data.find("sesuai")] + " "
    
    return " ".join(data.split()[3:]).replace("“", "").replace("”", "").replace("\"", "").strip()

for filename in list_file:
    data = ""
    
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_putusan_tp(data))
    
        
    
    

membawa pergi seorang wanita yang belum dewasa
dengan sengaja, membujuk anak untuk melakukan persetubuhan dengannya
pencurian
penggelapan
karena kelalaiannya mengemudikan kendaraan bermotor mengakibatkan orang lain meninggal dunia;
pencurian dalam keadaan memberatkan
dengan terang-terangan dan tenaga bersama menggunakan kekerasan terhadap orang yang mengakibatkan luka-luka
pencurian dalam keadaan memberatkan
dengan sengaja membujuk anak bersetubuh dengannya dilakukan oleh tenaga kependidikan secara berlanjut


In [10]:
def cari_kuhp_tp(data):
    data = " ".join(data.split()).lower()
    start = data.find("setelah mendengar pembacaan tuntutan pidana yang diajukan oleh penuntut umum yang pada pokoknya sebagai berikut:")
    data = data[start:]
    
    pattern = [
        "pasal",
        "sebagaimana diatur dan diancam dalam",
    ]
    
    start = -1
    
    for pat in pattern:
        if data.find(pat) != -1:
            data = data[data.find(pat):] + " "
            data = data[:data.find("2.")] + " "
            data = data[:data.find(";")] + " "
            data = data[:data.find("tentang")] + " "
            data = data.strip()
            break

    
    return data

    
for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_kuhp_tp(data))

pasal 83 jo. 76 f undang-undang nomor 35 tahun 2014
pasal 81 ayat (2) undang-undang nomor 17 tahun 2016
pasal pasal 362 kuhpidana
pasal 372 kitab undang-undang hukum pidana sebagaimana dakwaan alternatif pertama penuntut umum.
pasal 310 ayat (4) undang-undang republik indonesia nomor 22 tahun 2009
pasal 363 ayat (2) kuhp dalam surat dakwaan kami
pasal 170 ayat (2) ke-1 kuhpidana
pasal 363 ayat (1) ke-3 kuhpidana
pasal 81 ayat (2), ayat (3) undang undang nomor 17 tahun 2016 jo. undang undang nomor 35 tahun 2014


In [11]:
def cari_hukuman_tp(data):
    data = " ".join(data.split()).lower()
    start = data.find("setelah mendengar pembacaan tuntutan pidana yang diajukan oleh penuntut umum yang pada pokoknya sebagai berikut:")
    data = data[start:]
    
    start = data.find("selama")
    end = data.find("3.")
    hasil = data[start:end].strip()
  
    if hasil[-1] in ".;":
        hasil = hasil[:-1]
        
    return hasil

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_hukuman_tp(data))

selama 1 (satu) tahun dan 6 (enam) bulan dikurangi selama terdakwa tersebut berada dalam tahanan dengan perintah agar terdakwa tetap ditahan
selama 10 (sepuluh) tahun dikurangi selama terdakwa tersebut berada dalam tahanan dengan perintah agar terdakwa tetap ditahan
selama 1 (satu) tahun 6 (enam) bulan penjara dikurangi dengan masa penahanan sementara yang telah dijalani oleh terdakwa dengan perintah terdakwa tetap ditahan
selama 2 (dua) tahun dikurangkan selama terdakwa berada dalam tahanan dengan perintah agar terdakwa tetap ditahan
selama 1 (satu) tahun dikurangi selama terdakwa tersebut berada dalam tahanan dengan perintah agar terdakwa tetap ditahan
selama 1 (satu) tahun dan 6 (enam) bulan dikurangi selama terdakwa berada dalam tahanan dengan perintah agar terdakwa tetap ditahan
selama 11 (sebelas) bulan , dikurangi selama para terdakwa tersebut berada dalam tahanan dengan perintah agar para terdakwa tetap ditahan
selama 2 (dua) tahun dikurangi masa penahanan yang telah dijalani d

In [132]:
def cari_hukuman_p(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("selama")
    end = data.find("3. ")
    hasil = data[start:end] + " "
    hasil = hasil[:hasil.find(";")]
  
        
    return hasil

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_hukuman_p(data))

FileNotFoundError: [Errno 2] No such file or directory: './INPUT/zaf0379ca1d95c3ab511313331303331.txt'

In [201]:
def cari_hakim(data):
    data = " ".join(data.split())
    

    mengadili = [
        "MENGADILI:",
        "M E N G A D I L I:",
        "M E N G A D I L I :",
        "M E N G A D I L I"
    ]
    
    # Memulai pencarian setelah kata MENGADI
    start = -1
    
    for i in mengadili:
        if start != -1:
            break
      
        start = data.find(i)
   
    data = data[start:].lower()
    data = data[data.find("6."):]
    data = data[data.find("oleh kami") + len("oleh kami") if data.find("oleh kami") != -1 else data.find("oleh") + len("oleh"):].strip()
    print(data)

    if data[0] == ",":
        data = data[2:]
    

    data_hakim = {
        
    }
    
    data_hakim["hakim ketua"] = data[:data.find("sebagai hakim")].strip()
    
    if data_hakim["hakim ketua"][-1] == ",":
        data_hakim["hakim ketua"] = data_hakim["hakim ketua"][:-1]
    
    hakim_anggota = data[data.find("sebagai hakim ketua") + len("sebagai hakim ketua"):]
    hakim_anggota = hakim_anggota[:hakim_anggota.find("masing")].strip().split("dan")

    for hakim in range(len(hakim_anggota)):
        if hakim_anggota[hakim][0] == ",":
            hakim_anggota[hakim] = hakim_anggota[hakim][1:].strip()
        
        if hakim_anggota[hakim][-1] == ",":
            hakim_anggota[hakim] = hakim_anggota[hakim][:-1].strip()
        
            
    data_hakim["hakim anggota"] = hakim_anggota
    return data_hakim

list_file = os.listdir("./INPUT/")

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_hakim(data))

, agus maksum mulyohadi, s.h., m.h., sebagai hakim ketua , nofan hidayat, s.h., m.h., dan budi setyawan, s.h. masing-masing sebagai hakim anggota, yang diucapkan dalam sidang terbuka untuk umum pada hari rabu. tanggal 20 maret 2024. oleh hakim ketua dengan didampingi para hakim anggota tersebut, dibantu oleh siroju munir, s.h., m.h., panitera pengganti pada pengadilan negeri pekalongan, serta dihadiri oleh eko hertanto, s.h.,m.h., penuntut umum dan terdakwa didampingi oleh penasihat hukumnya; hakim anggota, hakim ketua, nofan hidayat, s.h., m.h. agus maksum mulyohadi, s.h., m.h. budi setyawan, s.h. panitera pengganti, siroju munir, s.h., m.h. halaman 19 dari 19 putusan nomor 17/pid.sus/2024/pn pkl disclaimer kepaniteraan mahkamah agung republik indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen mahkamah agung untuk pelayanan publik, transparansi dan akuntabilitas pelaksanaan fungsi peradilan. namun dalam hal-hal tertentu masih dimungki

In [14]:
def cari_tanggal_putusan(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    data = data[data.find("tanggal") + len("tanggal"):]
    data = data[:data.find(",")].strip()
    return data
    
for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_tanggal_putusan(data))

18 maret 2024
14 maret 2024
13 maret 2024
7 maret 2024
5 maret 2024
…..
02 april 2024 oleh kami
2 april 2024
1 april 2024


In [15]:
def cari_panitera(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    data = data[data.find("dibantu oleh") + len("dibantu oleh "):]
    data = data[:data.find("panitera")].strip()
    return data

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_panitera(data))

siroju munir, s.h., m.h.,
m. evans firmansyah, s.h.,
siroju munir, s.h., m.h.,
m. evans firmansyah, s.h.,
siroju munir, s.h., m.h.,
subagyo, s.h.,
subagyo, s.h.,
m. evans firmansyah, s.h.
m. evans firmansyah, s.h.,


In [16]:
def cari_penuntut(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    data = data[data.find("dihadiri oleh") + len("dihadiri oleh "):]
    data = data[:data.find("penuntut")].strip()
    
    return data

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_penuntut(data))

eko hertanto, s.h.,m.h.,
eko hertanto, s.h., m.h.,
ida nurliana, s.h.,
muhammad isa yeihansyah, s.h.,m.h.,
eko hertanto, s.h.,m.h.,
ida nurliana, s.h.
eko hertanto, s.h., m.h.,
broto susilo, s.h. , m.h.,
maziyah, s.h.,


In [17]:
def cari_nomor_putusan(data):
    start_index = data.find("Nomor")
    end_index = data.find("DEMI")
    return data[start_index + len("Nomor"):end_index].strip()

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_nomor_putusan(data))

XX/Pid.Sus/2024/PN Pkl
XXX/Pid.Sus/2023/PN Pkl
36/Pid.B/2024/PN Pkl
9/Pid.B/2024/PN Pkl
12/Pid.Sus/2024/PN Pkl
25/Pid.B/2024/PN Pkl
42/Pid.B/2024/PN Pkl
22/Pid.B/2024/PN Pkl
14/Pid.Sus/2024/PN Pkl


In [102]:
def generateEntity(fileName):
    input_folder = "./INPUT/"
    output_folder = "./OUTPUT/"
    
    with open(output_folder + "O-" + fileName, "w", encoding="utf-8") as output_file:

        with open(input_folder + fileName, "r", encoding="utf-8") as input_file:
            data = input_file.read()
            nomor_putusan = cari_nomor_putusan(data)
            terdakwa = cari_terdakwa(data)
            
            tuntutan_tp = cari_tuntutan_tp(data)
            kuhp = cari_kuhp_tp(data)
            hukuman = cari_hukuman_tp(data)
            
            putusan_tp = cari_putusan_tp(data)
            hukuman_p = cari_hukuman_p(data)
            tanggal = cari_tanggal_putusan(data)
            hakim = cari_hakim(data)
            panitera = cari_panitera(data)
            penuntut = cari_penuntut(data)

            
            print("Nomor Putusan: " + nomor_putusan)
            output_file.writelines("Nomor Putusan: " + nomor_putusan + "\n")
            for terdakwa_index in range(len(terdakwa)):
                print("Terdakwa Ke: " + str(terdakwa_index + 1))
                output_file.writelines("Terdakwa Ke: " + str(terdakwa_index + 1) + "\n")
                print("Nama Terdakwa: " + terdakwa[terdakwa_index])
                output_file.writelines("Nama Terdakwa: " + terdakwa[terdakwa_index] + "\n" + "\n")
                print()

            print("TUNTUTAN")
            output_file.writelines("TUNTUTAN" + "\n")
            print("Tindak Pidana: " + tuntutan_tp )
            output_file.writelines("Tindak Pidana: " + tuntutan_tp + "\n")
            print("Melanggar KUHP: " + kuhp)
            output_file.writelines("Melanggar KUHP: " + kuhp + "\n")
            print("Tuntutan Pidana: " + hukuman)
            output_file.writelines("Tuntutan Pidana: " + hukuman + "\n" + "\n")
            
            print()
            print("PUTUSAN")
            output_file.writelines("PUTUSAN" + "\n")
            print("Tindak Pidana: " + putusan_tp )
            output_file.writelines("Tindak Pidana: " + putusan_tp + "\n")
            print("Hukunan Pidana: " + hukuman_p)
            output_file.writelines("Hukunan Pidana: " + hukuman_p + "\n")
            print("Tanggal Putusan: " + tanggal)
            output_file.writelines("Tanggal Putusan: " + tanggal + "\n"  + "\n")
            
            print()
            print("Hakim Ketua: " + hakim["hakim ketua"])
            output_file.writelines("Hakim Ketua: " + hakim["hakim ketua"] + "\n")
            
            for h in range(len(hakim["hakim anggota"])):
                print("Hakim Anggota " + str(h + 1) + ": ", hakim["hakim anggota"][h])
                output_file.writelines("Hakim Anggota " + str(h + 1) + ": " + hakim["hakim anggota"][h] + "\n")
                
            print("Panitera: " + panitera)
            output_file.writelines("Panitera: " + panitera + "\n")
            print("Penuntut Umum: " + penuntut)
            output_file.writelines("Penuntut Umum: " + penuntut + "\n")
            
         
    

In [ ]:
import os
# from courtHistoryCsv import courtHistory

def main():

    start_path = "./INPUT/"
    hasil_path = "./OUTPUT/"
    daftarFile = os.listdir(start_path)
    listHasil = []


    for tiapFile in daftarFile:
        print("======= ROW HASIL =======")    
        with open(start_path + tiapFile, "r", encoding="utf-8") as file:
            hasil = generateEntity(file.read())
        print("======= ROW AKHIR =======")
        print()
 

main()


======= ROW HASIL =======
Nomor Putusan: XX/Pid.Sus/2024/PN Pkl
Terdakwa Ke: 1
Nama Terdakwa: terdakwa

TUNTUTAN
Tindak Pidana: membawa pergi seorang wanita yang belum dewasa, tanpa dikehendaki orang tuanya atau walinya tetapi dengan persetujuannya
Melanggar KUHP: pasal 83 jo. 76 f undang-undang nomor 35 tahun 2014
Tuntutan Pidana: selama 1 (satu) tahun dan 6 (enam) bulan dikurangi selama terdakwa tersebut berada dalam tahanan dengan perintah agar terdakwa tetap ditahan

PUTUSAN
Tindak Pidana: membawa pergi seorang wanita yang belum dewasa
Hukunan Pidana: selama 1 (satu) tahun
Tanggal Putusan: 18 maret 2024

Hakim Ketua: agus maksum mulyohadi, s.h., m.h.
Hakim Anggota 1:  nofan hidayat, s.h., m.h.
Hakim Anggota 2:   budi setyawan, s.h.
Panitera: siroju munir, s.h., m.h.,
Penuntut Umum: eko hertanto, s.h.,m.h.,
======= ROW AKHIR =======

======= ROW HASIL =======
Nomor Putusan: XXX/Pid.Sus/2023/PN Pkl
Terdakwa Ke: 1
Nama Terdakwa: xxxxxx

TUNTUTAN
Tindak Pidana: dengan sengaja melakukan